# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [481]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Ask for parameters

In [482]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [483]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

* **t:** Time period (t = 1, 2, ..., 12)
* **f:** Food product (f = 1, 2)
* **b:** Beverage product (b = 1, 2, 3)

## Parameters:

* **FC_f,t:** Fixed cost of producing one unit of food f in time period t
* **VC_f,t:** Variable cost of producing one unit of food f in time period t
* **FC_b,t:** Fixed cost of producing one liter of beverage b in time period t
* **VC_b,t:** Variable cost of producing one liter of beverage b in time period t
* **SC_f,t:** Variable cost of storing one unit of food f in time period t
* **SC_b,t:** Variable cost of storing one liter of beverage b in time period t
* **D_f,t:** Demand for food f in time period t
* **D_b,t:** Demand for beverage b in time period t
* **R_f,t:** Revenue for selling one unit of food f in time period t
* **R_b,t:** Revenue for selling one liter of beverage b in time period t
* **StorageCap_f:** Storage capacity for food f 
* **StorageCap_b:** Storage capacity for beverage b
* **ProdCap_f:** Production capacity for food f
* **ProdCap_b:** Production capacity for beverage b

## Decision Variables:

* **X_f,t:** Quantity of food f produced in time period t (units, integer)
* **X_b,t:** Quantity of beverage b produced in time period t (liters, continuous)
* **I_f,t:** Quantity of food f stored at the end of time period t (units, integer)
* **I_b,t:** Quantity of beverage b stored at the end of time period t (liters, continuous)
* **S_f,t:** Quantity of food f sold in time period t (units, integer)
* **S_b,t:** Quantity of beverage b sold in time period t (liters, continuous) 
 


# 2. Ask for objective

In [484]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [485]:
Markdown(response2.text)

\begin{align*}
\text{Maximize} \quad & \sum_{t=1}^{12} \left( \sum_{f=1}^{2} (R_{f,t}S_{f,t} - FC_{f,t}X_{f,t} - VC_{f,t}X_{f,t} - SC_{f,t}I_{f,t})  + \sum_{b=1}^{3} (R_{b,t}S_{b,t} - FC_{b,t}X_{b,t} - VC_{b,t}X_{b,t} - SC_{b,t}I_{b,t}) \right)
\end{align*} 


# 3. Ask for constraints

In [486]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [487]:
Markdown(response3.text)

## Constraints:

**1. Production Capacity:**
  *  0 ≤ X_f,t ≤ ProdCap_f  for all f = 1, 2 and t = 1, 2, ..., 12 
  *  0 ≤ X_b,t ≤ ProdCap_b  for all b = 1, 2, 3 and t = 1, 2, ..., 12 

**2. Storage Capacity:**
  *  0 ≤ I_f,t ≤ StorageCap_f  for all f = 1, 2 and t = 1, 2, ..., 12
  *  0 ≤ I_b,t ≤ StorageCap_b  for all b = 1, 2, 3 and t = 1, 2, ..., 12 

**3. Demand Fulfillment (Maximum Sales):**
  *  S_f,t ≤ D_f,t for all f = 1, 2 and t = 1, 2, ..., 12
  *  S_b,t ≤ D_b,t for all b = 1, 2, 3 and t = 1, 2, ..., 12 

**4. Inventory Balance:**
  *  I_f,t = I_f,t-1 + X_f,t - S_f,t  for all f = 1, 2 and t = 1, 2, ..., 12  (Assuming I_f,0 = 0 for all f)
  *  I_b,t = I_b,t-1 + X_b,t - S_b,t  for all b = 1, 2, 3 and t = 1, 2, ..., 12 (Assuming I_b,0 = 0 for all b)

**5. Non-Negativity:**
  *  X_f,t, I_f,t, S_f,t ≥ 0  for all f = 1, 2 and t = 1, 2, ..., 12
  *  X_b,t, I_b,t, S_b,t ≥ 0  for all b = 1, 2, 3 and t = 1, 2, ..., 12

**6. Integer Constraints:**
  *  X_f,t, I_f,t, S_f,t ∈ Z (integers) for all f = 1, 2 and t = 1, 2, ..., 12 


## 3. Generate the pyomo code

In [488]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [489]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
# Parameters
FC_f = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  # Example: Fixed cost for food
VC_f = {(1, 1): 10, (1, 2): 11, (2, 1): 12, (2, 2): 13}   # Example: Variable cost for food
FC_b = {(1, 1): 50, (1, 2): 55, (2, 1): 60, (2, 2): 65}    # Example: Fixed cost for beverage
VC_b = {(1, 1): 5, (1, 2): 6, (2, 1): 7, (2, 2): 8}     # Example: Variable cost for beverage 
SC_f = {(1, 1): 2, (1, 2): 3, (2, 1): 4, (2, 2): 5}      # Example: Storage cost for food
SC_b = {(1, 1): 1, (1, 2): 2, (2, 1): 3, (2, 2): 4}      # Example: Storage cost for beverage
D_f = {(1, 1): 200, (1, 2): 220, (2, 1): 240, (2, 2): 260} # Example: Demand for food
D_b = {(1, 1): 300, (1, 2): 330, (2, 1): 360, (2, 2): 390} # Example: Demand for beverage
R_f = {(1, 1): 30, (1, 2): 33, (2, 1): 36, (2, 2): 39}   # Example: Revenue for food
R_b = {(1, 1): 20, (1, 2): 22, (2, 1): 24, (2, 2): 26}    # Example: Revenue for beverage

StorageCap_f = {1: 1000, 2: 1200}  # Example: Storage capacity for food 
StorageCap_b = {1: 1500, 2: 1800}  # Example: Storage capacity for beverage 
ProdCap_f = {1: 500, 2: 600}         # Example: Production capacity for food
ProdCap_b = {1: 700, 2: 800}         # Example: Production capacity for beverage

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.RangeSet(1, 2)  # Food products
model.B = pyo.RangeSet(1, 3)  # Beverage products

# Parameters (using sample data)
model.FC_f = pyo.Param(model.F, model.T, initialize=FC_f, default=0) 
model.VC_f = pyo.Param(model.F, model.T, initialize=VC_f, default=0)
model.FC_b = pyo.Param(model.B, model.T, initialize=FC_b, default=0)
model.VC_b = pyo.Param(model.B, model.T, initialize=VC_b, default=0)
model.SC_f = pyo.Param(model.F, model.T, initialize=SC_f, default=0)
model.SC_b = pyo.Param(model.B, model.T, initialize=SC_b, default=0)
model.D_f = pyo.Param(model.F, model.T, initialize=D_f, default=0)
model.D_b = pyo.Param(model.B, model.T, initialize=D_b, default=0)
model.R_f = pyo.Param(model.F, model.T, initialize=R_f, default=0)
model.R_b = pyo.Param(model.B, model.T, initialize=R_b, default=0)
model.StorageCap_f = pyo.Param(model.F, initialize=StorageCap_f)
model.StorageCap_b = pyo.Param(model.B, initialize=StorageCap_b)
model.ProdCap_f = pyo.Param(model.F, initialize=ProdCap_f)
model.ProdCap_b = pyo.Param(model.B, initialize=ProdCap_b)

# Decision Variables
model.X_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.X_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.I_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.I_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return pyo.summation(model.R_f, model.S_f) + pyo.summation(model.R_b, model.S_b) - \
           pyo.summation(model.FC_f, model.X_f) - pyo.summation(model.VC_f, model.X_f) - \
           pyo.summation(model.FC_b, model.X_b) - pyo.summation(model.VC_b, model.X_b) - \
           pyo.summation(model.SC_f, model.I_f) - pyo.summation(model.SC_b, model.I_b)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def production_capacity_f_rule(model, f, t):
    return model.X_f[f, t] <= model.ProdCap_f[f]
model.production_capacity_f = pyo.Constraint(model.F, model.T, rule=production_capacity_f_rule)

def production_capacity_b_rule(model, b, t):
    return model.X_b[b, t] <= model.ProdCap_b[b]
model.production_capacity_b = pyo.Constraint(model.B, model.T, rule=production_capacity_b_rule)

def storage_capacity_f_rule(model, f, t):
    return model.I_f[f, t] <= model.StorageCap_f[f]
model.storage_capacity_f = pyo.Constraint(model.F, model.T, rule=storage_capacity_f_rule)

def storage_capacity_b_rule(model, b, t):
    return model.I_b[b, t] <= model.StorageCap_b[b]
model.storage_capacity_b = pyo.Constraint(model.B, model.T, rule=storage_capacity_b_rule)

def demand_fulfillment_f_rule(model, f, t):
    return model.S_f[f, t] <= model.D_f[f, t]
model.demand_fulfillment_f = pyo.Constraint(model.F, model.T, rule=demand_fulfillment_f_rule)

def demand_fulfillment_b_rule(model, b, t):
    return model.S_b[b, t] <= model.D_b[b, t]
model.demand_fulfillment_b = pyo.Constraint(model.B, model.T, rule=demand_fulfillment_b_rule)

def inventory_balance_f_rule(model, f, t):
    if t == 1:
        return model.I_f[f, t] == model.X_f[f, t] - model.S_f[f, t]
    else:
        return model.I_f[f, t] == model.I_f[f, t-1] + model.X_f[f, t] - model.S_f[f, t] 
model.inventory_balance_f = pyo.Constraint(model.F, model.T, rule=inventory_balance_f_rule)

def inventory_balance_b_rule(model, b, t):
    if t == 1:
        return model.I_b[b, t] == model.X_b[b, t] - model.S_b[b, t]
    else:
        return model.I_b[b, t] == model.I_b[b, t-1] + model.X_b[b, t] - model.S_b[b, t]
model.inventory_balance_b = pyo.Constraint(model.B, model.T, rule=inventory_balance_b_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Use CBC solver (replace with other solvers if needed)
solver.solve(model)

# Print the results
print("Profit = ", pyo.value(model.profit))

# You can access the values of decision variables using model.X_f[f,t].value, etc. 
```


## 4. Input problem data and try running the generated code

In [495]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
# Parameters
FC_f = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  # Example: Fixed cost for food
VC_f = {(1, 1): 10, (1, 2): 11, (2, 1): 12, (2, 2): 13}   # Example: Variable cost for food
FC_b = {(1, 1): 50, (1, 2): 55, (2, 1): 60, (2, 2): 65}    # Example: Fixed cost for beverage
VC_b = {(1, 1): 5, (1, 2): 6, (2, 1): 7, (2, 2): 8}     # Example: Variable cost for beverage 
SC_f = {(1, 1): 2, (1, 2): 3, (2, 1): 4, (2, 2): 5}      # Example: Storage cost for food
SC_b = {(1, 1): 1, (1, 2): 2, (2, 1): 3, (2, 2): 4}      # Example: Storage cost for beverage
D_f = {(1, 1): 200, (1, 2): 220, (2, 1): 240, (2, 2): 260} # Example: Demand for food
D_b = {(1, 1): 300, (1, 2): 330, (2, 1): 360, (2, 2): 390} # Example: Demand for beverage
R_f = {(1, 1): 30, (1, 2): 33, (2, 1): 36, (2, 2): 39}   # Example: Revenue for food
R_b = {(1, 1): 20, (1, 2): 22, (2, 1): 24, (2, 2): 26}    # Example: Revenue for beverage

StorageCap_f = {1: 1000, 2: 1200}  # Example: Storage capacity for food 
StorageCap_b = {1: 1500, 2: 1800, 3:22}  # Example: Storage capacity for beverage 
ProdCap_f = {1: 500, 2: 600}         # Example: Production capacity for food
ProdCap_b = {1: 700, 2: 800, 3:400}         # Example: Production capacity for beverage


##MANUAL START

fixed_cost_production = pd.read_csv("fixed_cost_production.csv", index_col=[0])
variable_cost_production = pd.read_csv("variable_cost_production.csv", index_col=[0])
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", index_col=[0])
demand = pd.read_csv("demand.csv", index_col=[0])
revenue = pd.read_csv("revenue.csv", index_col=[0])



# Function to convert DataFrame to the desired dictionary format
def convert_to_dict(df, item_type):
    if item_type == "food":
        data = df.iloc[:2, :]  # Select first two rows for food
    elif item_type == "beverage":
        data = df.iloc[2:, :]  # Select last three rows for beverage

    result_dict = {(i+1, t+1): data.iloc[i, t] for i in range(data.shape[0]) for t in range(data.shape[1])}
    return result_dict

# Convert each DataFrame to the desired dictionary format
fixed_cost_production_food = convert_to_dict(fixed_cost_production, "food")
fixed_cost_production_beverage = convert_to_dict(fixed_cost_production, "beverage")

variable_cost_production_food = convert_to_dict(variable_cost_production, "food")
variable_cost_production_beverage = convert_to_dict(variable_cost_production, "beverage")

variable_cost_storage_food = convert_to_dict(variable_cost_storage, "food")
variable_cost_storage_beverage = convert_to_dict(variable_cost_storage, "beverage")

demand_food = convert_to_dict(demand, "food")
demand_beverage = convert_to_dict(demand, "beverage")

revenue_food = convert_to_dict(revenue, "food")
revenue_beverage = convert_to_dict(revenue, "beverage")
###END
C_f = {1: 580, 2: 687}  
C_b = {1:599, 2:788, 3:294}
P_f = {1: 1080, 2: 908} 
P_b = {1: 408, 2: 1000,3:403} 


FC_f = fixed_cost_production_food  # Example: Fixed cost for food
VC_f = variable_cost_production_food   # Example: Variable cost for food
FC_b = fixed_cost_production_beverage    # Example: Fixed cost for beverage
VC_b = variable_cost_production_beverage     # Example: Variable cost for beverage 
SC_f = variable_cost_storage_food      # Example: Storage cost for food
SC_b = variable_cost_storage_beverage      # Example: Storage cost for beverage
D_f = demand_food # Example: Demand for food
D_b = demand_beverage # Example: Demand for beverage
R_f = revenue_food  # Example: Revenue for food
R_b = revenue_beverage    # Example: Revenue for beverage

StorageCap_f = {1: 580, 2: 687}  # Example: Storage capacity for food 
StorageCap_b = {1:599, 2:788, 3:294}  # Example: Storage capacity for beverage 
ProdCap_f = {1: 1080, 2: 908}          # Example: Production capacity for food
ProdCap_b =  {1: 408, 2: 1000,3:403}         # Example: Production capacity for beverage


#MANUAL END







# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.RangeSet(1, 2)  # Food products
model.B = pyo.RangeSet(1, 3)  # Beverage products

# Parameters (using sample data)
model.FC_f = pyo.Param(model.F, model.T, initialize=FC_f, default=0) 
model.VC_f = pyo.Param(model.F, model.T, initialize=VC_f, default=0)
model.FC_b = pyo.Param(model.B, model.T, initialize=FC_b, default=0)
model.VC_b = pyo.Param(model.B, model.T, initialize=VC_b, default=0)
model.SC_f = pyo.Param(model.F, model.T, initialize=SC_f, default=0)
model.SC_b = pyo.Param(model.B, model.T, initialize=SC_b, default=0)
model.D_f = pyo.Param(model.F, model.T, initialize=D_f, default=0)
model.D_b = pyo.Param(model.B, model.T, initialize=D_b, default=0)
model.R_f = pyo.Param(model.F, model.T, initialize=R_f, default=0)
model.R_b = pyo.Param(model.B, model.T, initialize=R_b, default=0)
model.StorageCap_f = pyo.Param(model.F, initialize=StorageCap_f)
model.StorageCap_b = pyo.Param(model.B, initialize=StorageCap_b)
model.ProdCap_f = pyo.Param(model.F, initialize=ProdCap_f)
model.ProdCap_b = pyo.Param(model.B, initialize=ProdCap_b)

# Decision Variables
model.X_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.X_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.I_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.I_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return pyo.summation(model.R_f, model.S_f) + pyo.summation(model.R_b, model.S_b) - \
           pyo.summation(model.FC_f, model.X_f) - pyo.summation(model.VC_f, model.X_f) - \
           pyo.summation(model.FC_b, model.X_b) - pyo.summation(model.VC_b, model.X_b) - \
           pyo.summation(model.SC_f, model.I_f) - pyo.summation(model.SC_b, model.I_b)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def production_capacity_f_rule(model, f, t):
    return model.X_f[f, t] <= model.ProdCap_f[f]
model.production_capacity_f = pyo.Constraint(model.F, model.T, rule=production_capacity_f_rule)

def production_capacity_b_rule(model, b, t):
    return model.X_b[b, t] <= model.ProdCap_b[b]
model.production_capacity_b = pyo.Constraint(model.B, model.T, rule=production_capacity_b_rule)

def storage_capacity_f_rule(model, f, t):
    return model.I_f[f, t] <= model.StorageCap_f[f]
model.storage_capacity_f = pyo.Constraint(model.F, model.T, rule=storage_capacity_f_rule)

def storage_capacity_b_rule(model, b, t):
    return model.I_b[b, t] <= model.StorageCap_b[b]
model.storage_capacity_b = pyo.Constraint(model.B, model.T, rule=storage_capacity_b_rule)

def demand_fulfillment_f_rule(model, f, t):
    return model.S_f[f, t] <= model.D_f[f, t]
model.demand_fulfillment_f = pyo.Constraint(model.F, model.T, rule=demand_fulfillment_f_rule)

def demand_fulfillment_b_rule(model, b, t):
    return model.S_b[b, t] <= model.D_b[b, t]
model.demand_fulfillment_b = pyo.Constraint(model.B, model.T, rule=demand_fulfillment_b_rule)

def inventory_balance_f_rule(model, f, t):
    if t == 1:
        return model.I_f[f, t] == model.X_f[f, t] - model.S_f[f, t]
    else:
        return model.I_f[f, t] == model.I_f[f, t-1] + model.X_f[f, t] - model.S_f[f, t] 
model.inventory_balance_f = pyo.Constraint(model.F, model.T, rule=inventory_balance_f_rule)

def inventory_balance_b_rule(model, b, t):
    if t == 1:
        return model.I_b[b, t] == model.X_b[b, t] - model.S_b[b, t]
    else:
        return model.I_b[b, t] == model.I_b[b, t-1] + model.X_b[b, t] - model.S_b[b, t]
model.inventory_balance_b = pyo.Constraint(model.B, model.T, rule=inventory_balance_b_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Use CBC solver (replace with other solvers if needed)
solver.solve(model)

# Print the results
print("Profit = ", pyo.value(model.profit))

Profit =  508.19059999999996


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [497]:
print(response.text)

## Indices:

* **t:** Time period (t = 1, 2, ..., 12)
* **f:** Food product (f = 1, 2)
* **b:** Beverage product (b = 1, 2, 3)

## Parameters:

* **FC_f,t:** Fixed cost of producing one unit of food f in time period t
* **VC_f,t:** Variable cost of producing one unit of food f in time period t
* **FC_b,t:** Fixed cost of producing one liter of beverage b in time period t
* **VC_b,t:** Variable cost of producing one liter of beverage b in time period t
* **SC_f,t:** Variable cost of storing one unit of food f in time period t
* **SC_b,t:** Variable cost of storing one liter of beverage b in time period t
* **D_f,t:** Demand for food f in time period t
* **D_b,t:** Demand for beverage b in time period t
* **R_f,t:** Revenue for selling one unit of food f in time period t
* **R_b,t:** Revenue for selling one liter of beverage b in time period t
* **StorageCap_f:** Storage capacity for food f 
* **StorageCap_b:** Storage capacity for beverage b
* **ProdCap_f:** Production capacity for food

In [498]:
print(response2.text)

\begin{align*}
\text{Maximize} \quad & \sum_{t=1}^{12} \left( \sum_{f=1}^{2} (R_{f,t}S_{f,t} - FC_{f,t}X_{f,t} - VC_{f,t}X_{f,t} - SC_{f,t}I_{f,t})  + \sum_{b=1}^{3} (R_{b,t}S_{b,t} - FC_{b,t}X_{b,t} - VC_{b,t}X_{b,t} - SC_{b,t}I_{b,t}) \right)
\end{align*} 



In [499]:
print(response3.text)

## Constraints:

**1. Production Capacity:**
  *  0 ≤ X_f,t ≤ ProdCap_f  for all f = 1, 2 and t = 1, 2, ..., 12 
  *  0 ≤ X_b,t ≤ ProdCap_b  for all b = 1, 2, 3 and t = 1, 2, ..., 12 

**2. Storage Capacity:**
  *  0 ≤ I_f,t ≤ StorageCap_f  for all f = 1, 2 and t = 1, 2, ..., 12
  *  0 ≤ I_b,t ≤ StorageCap_b  for all b = 1, 2, 3 and t = 1, 2, ..., 12 

**3. Demand Fulfillment (Maximum Sales):**
  *  S_f,t ≤ D_f,t for all f = 1, 2 and t = 1, 2, ..., 12
  *  S_b,t ≤ D_b,t for all b = 1, 2, 3 and t = 1, 2, ..., 12 

**4. Inventory Balance:**
  *  I_f,t = I_f,t-1 + X_f,t - S_f,t  for all f = 1, 2 and t = 1, 2, ..., 12  (Assuming I_f,0 = 0 for all f)
  *  I_b,t = I_b,t-1 + X_b,t - S_b,t  for all b = 1, 2, 3 and t = 1, 2, ..., 12 (Assuming I_b,0 = 0 for all b)

**5. Non-Negativity:**
  *  X_f,t, I_f,t, S_f,t ≥ 0  for all f = 1, 2 and t = 1, 2, ..., 12
  *  X_b,t, I_b,t, S_b,t ≥ 0  for all b = 1, 2, 3 and t = 1, 2, ..., 12

**6. Integer Constraints:**
  *  X_f,t, I_f,t, S_f,t ∈ Z (integers) f

In [500]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
# Parameters
FC_f = {(1, 1): 100, (1, 2): 110, (2, 1): 120, (2, 2): 130}  # Example: Fixed cost for food
VC_f = {(1, 1): 10, (1, 2): 11, (2, 1): 12, (2, 2): 13}   # Example: Variable cost for food
FC_b = {(1, 1): 50, (1, 2): 55, (2, 1): 60, (2, 2): 65}    # Example: Fixed cost for beverage
VC_b = {(1, 1): 5, (1, 2): 6, (2, 1): 7, (2, 2): 8}     # Example: Variable cost for beverage 
SC_f = {(1, 1): 2, (1, 2): 3, (2, 1): 4, (2, 2): 5}      # Example: Storage cost for food
SC_b = {(1, 1): 1, (1, 2): 2, (2, 1): 3, (2, 2): 4}      # Example: Storage cost for beverage
D_f = {(1, 1): 200, (1, 2): 220, (2, 1): 240, (2, 2): 260} # Example: Demand for food
D_b = {(1, 1): 300, (1, 2): 330, (2, 1): 360, (2, 2): 390} # Example: Demand for beverage
R_f = {(1, 1): 30, (1, 2): 33, (2, 1): 36, (2, 2): 39}   # Example: Revenue for food
R_b = {(1, 1): 20, (1, 2): 22, (2, 1): 24, (2, 2): 26}    # Example: Revenue for beverage